In [ ]:
import copy

import numpy
from IPython.display import display

from rmgpy.tools.uncertainty import Uncertainty, ThermoParameterUncertainty, KineticParameterUncertainty

# Parameter Uncertainty Assignment

This script aims to extract model sources in a clear and informative format.  The script first shows what all the kinetic and thermo sources are in a model. Then it goes through each reaction and species to show their source and what the assigned uncertainties are.  This can be used with any RMG-generated CHEMKIN file that is annotated.

__Note__: The RMG-database version must match the version used to generate the model. RMG will attempt to recreate the kinetics estimate for each reaction and may fail if the database is different.

## Run RMG job
This cell is used only when the user is interested in learning this module with the provided toy model, ./data/parse_source/input.py.
The user can speed up the model generation by commenting out the uncertainty block in ./data/parse_source/input.py.

In [ ]:
!python ../rmg.py ./data/parse_source/input.py

## Perform parameter uncertainty assignment

In [ ]:
chem_file = './data/parse_source/chem_annotated.inp'
dict_file = './data/parse_source/species_dictionary.txt'

In [ ]:
uncertainty = Uncertainty(output_directory='./temp/uncertainty')

In [ ]:
uncertainty.load_model(chem_file, dict_file)

In [ ]:
# NOTE: You must load the database with the same settings which were used to generate the model.
#       This includes any thermo or kinetics libraries which were used.
uncertainty.load_database(
    thermo_libraries=['DFT_QCI_thermo', 'primaryThermoLibrary'],
    kinetics_families='default',
    reaction_libraries=[],
)

In [ ]:
uncertainty.extract_sources_from_model()
uncertainty.compile_all_sources()

In [ ]:
print('All Kinetic Sources')
for sourceType in uncertainty.all_kinetic_sources.keys():
    if sourceType == 'Library':
        print('============')
        print('Library kinetics')
        print('')
        print('\tReactions: ', uncertainty.all_kinetic_sources['Library'])
    elif sourceType == 'PDep':
        print('============')
        print('PDep kinetics')
        print('')
        print('\tReactions: ', uncertainty.all_kinetic_sources['PDep'])
    elif sourceType == 'Rate Rules':
        print('============')
        print('Rate rule kinetics')
        print('')
        for familyLabel, entries in uncertainty.all_kinetic_sources['Rate Rules'].items():
            print('\t', familyLabel)
            for entry in entries:
                print('\t\t', entry)
    elif sourceType == 'Training':
        print('============')
        print('Training reaction kinetics')
        print('')
        for familyLabel, entries in uncertainty.all_kinetic_sources['Training'].items():
            print('\t', familyLabel)
            for entry in entries:
                print('\t\t', entry)
    else:
        print(sourceType)
        raise Exception('Kinetics source must be Library, PDep, Rate Rules, or Training')

In [ ]:
print('All Thermo Sources')
for sourceType in uncertainty.all_thermo_sources.keys():
    if sourceType == 'Library':
        print('============')
        print('Library thermo')
        print('')
        print('\tSpecies: ', uncertainty.all_thermo_sources['Library'])
    elif sourceType == 'QM':
        print('============')
        print('QM thermo')
        print('')
        print('\tSpecies: ', uncertainty.all_thermo_sources['QM'])
    elif sourceType == 'GAV':
        print('============')
        print('Group additivity thermo')
        print('')
        for groupType, entries in uncertainty.all_thermo_sources['GAV'].items():
            print('\t', groupType)
            for entry in entries:
                print('\t\t', entry)
    else:
        raise Exception('Thermo source must be GAV, QM, or Library')

In [ ]:
# Assign all the uncertainties using the Uncertainty() class function
# ThermoParameterUncertainty and KineticParameterUncertainty classes may be customized and passed into this function
# if non-default constants for constructing the uncertainties are desired
uncertainty.assign_parameter_uncertainties()

In [ ]:
T = 623 # temperature in Kelvin for which to evaluate kinetics
P = 1e5  # Pa 

In [ ]:
g_param_engine = ThermoParameterUncertainty()
k_param_engine = KineticParameterUncertainty()

In [ ]:
for rxn, source in uncertainty.reaction_sources_dict.items():
    print('======')
    print(rxn)
    display(rxn)
    if 'Library' in source:
        print('Library reaction')
        print(source['Library'])
    elif 'PDep' in source:
        print('PDep reaction')
        print(source['PDep'])
    elif 'Rate Rules' in source:
        print('Rate rule estimate')
        family = source['Rate Rules'][0]
        sourceDict = source['Rate Rules'][1]
        originalTemplate = sourceDict['template']
        print('\tFamily = ', family)
        print('\tOriginal Template = ', [group.label for group in originalTemplate])
        print('\tExact = ', sourceDict['exact'])
        rules = sourceDict['rules']
        training = sourceDict['training']
        if rules:
            print('\tRate rule sources:')
            for ruleEntry, weight in rules:
                print('\t\t', ruleEntry, '=', weight)
        if training:
            print('\tTraining sources:')
            for ruleEntry, trainingEntry, weight in training:
                print('\t\t', ruleEntry , 'mapped to', trainingEntry , '=', weight)
    elif 'Training' in source:
        print('Training reaction')
        family = source['Training'][0]
        training = source['Training'][1]
        print('\t Family = ', family)
        print('\t\t', training)

    print('')
    print('Rate coefficient at {} K = {:.2e}'.format(T, rxn.kinetics.get_rate_coefficient(T,P)))

    # Uncomment the following lines if you want to verify that the parsing has been performed correctly by
    # checking the values for both the original and reconstructed kinetics
#     print('---------')
#     print('Original kinetics:')
#     print(rxn.kinetics)
#     print('')
#     print('Reconstructed kinetics from parsing:')
#     reconstructedKinetics = uncertainty.database.kinetics.reconstruct_kinetics_from_source(rxn, source, fix_barrier_height=True)
#     print(reconstructedKinetics)

#     rxnIndex = uncertainty.reaction_list.index(rxn)
#     print('Uncertainty dln(k) = ', uncertainty.kinetic_input_uncertainties[rxnIndex])
    
#     # Test that the partial uncertainty calculation is working
#     dlnk = 0.0
#     if 'Rate Rules' in source:
#         family = source['Rate Rules'][0]
#         sourceDict = source['Rate Rules'][1]
#         rules = sourceDict['rules']
#         training = sourceDict['training']
#         for ruleEntry, weight in rules:
#             dlnk += k_param_engine.get_partial_uncertainty_value(source, 'Rate Rules', corr_param=ruleEntry, corr_family=family)
#         for ruleEntry, trainingEntry, weight in training:
#             dlnk += k_param_engine.get_partial_uncertainty_value(source, 'Rate Rules', corr_param=ruleEntry, corr_family=family)
#         dlnk += k_param_engine.get_partial_uncertainty_value(source, 'Estimation')
#     elif 'PDep' in source:
#         dlnk += k_param_engine.get_partial_uncertainty_value(source, 'PDep', source['PDep'])
#     elif 'Library' in source:
#         dlnk += k_param_engine.get_partial_uncertainty_value(source, 'Library', source['Library'])
#     elif 'Training' in source:
#         dlnk += k_param_engine.get_partial_uncertainty_value(source, 'Training', source['Training'])
#     print('Uncertainty dlnk calculated using sum of partial values = ', dlnk)

In [ ]:
for species, source in uncertainty.species_sources_dict.items():
    print('==========')
    print(species)
    display(species)
    if 'Library' in source:
        print('Thermo Library: ', source['Library'])
    if 'QM' in source:
        print('QM: ', source['QM'])
    if 'GAV' in source:
        print('Group additivity:')
        for groupType, groupList in source['GAV'].items():
            print('\t', groupType)
            for group, weight in groupList:
                print('\t\t', group, '=', weight)

    spcIndex = uncertainty.species_list.index(species)    
    print('')
    print('Uncertainty dG = ', uncertainty.thermo_input_uncertainties[spcIndex], ' kcal/mol')
    
    
    # Test that the partial uncertainty calculation is working
    dG = 0.0
    if 'Library' in source:
        dG += g_param_engine.get_partial_uncertainty_value(source, 'Library', corr_param=source['Library'])
    if 'QM' in source:
        dG += g_param_engine.get_partial_uncertainty_value(source, 'QM',corr_param=source['QM'])
    if 'GAV' in source:
        for groupType, groupList in source['GAV'].items():
            for group, weight in groupList:
                dG += g_param_engine.get_partial_uncertainty_value(source, 'GAV', group, groupType)
        dG += g_param_engine.get_partial_uncertainty_value(source, 'Estimation')
    print('Uncertainty dG calculated using sum of partial values = ', dG, ' kcal/mol')

In [ ]:
# Assign correlated parameter uncertainties 
uncertainty.assign_parameter_uncertainties(correlated=True)

In [ ]:
# See the thermo correlated parameter partial uncertainties
uncertainty.thermo_input_uncertainties

In [ ]:
# See the kinetics correlated parameter partial uncertainties
uncertainty.kinetic_input_uncertainties